In [1]:
import requests
import pandas as pd

In [2]:
# define global variable needed for API
slotcodes = {
    0 : 'QB',
    2 : 'RB',
    4 : 'WR',
    6 : 'TE',
    16: 'Def',
    17: 'K',
    20: 'Bench',
    21: 'IR',
    23: 'Flex'
}

In [9]:
# get ESPN credentials
f = open('/Users/cbate/.ssh/espn', 'r')
lines = f.readlines()
swid = lines[0].strip('\n')
espn_s2 = lines[1].strip('\n')
f.close()

In [39]:
def PullESPN(season, league_id=901706, swid=None, espn_s2=None):
    # Pulls projections and results from ESPN for a given fantasy league and season. Season must be provided.

    # define API URL
    url = 'https://fantasy.espn.com/apis/v3/games/ffl/seasons/{0}/segments/0/leagues/{1}?view=mMatchup&view=mMatchupScore'.format(str(season), str(league_id))

    # iterate through each week of season
    data = []
    for week in range(1,17):
        # if private league, have to include swid and espn_s2
        if swid and espn_s2:
            response = requests.get(
                url,
                params = {'scoringPeriodId' : week},
                cookies = {'SWID' : swid, 'espn_s2' : espn_s2}
                ).json()
        # otherwise, do not need to include credentials in API call
        else:
            response = requests.get(
                url,
                params = {'scoringPeriodId' : week}
                ).json()

        # Iterate through teams and players to organize data and store in DataFrame
        for tm in response['teams']:
            tmid = tm['id']
            for p in tm['roster']['entries']:
                name = p['playerPoolEntry']['player']['fullName']
                lineupSlot = p['lineupSlotId']
                pos = slotcodes[lineupSlot]
                try:
                    inj = p['playerPoolEntry']['player']['injuryStatus']
                except: # this must be a DST, which doesn't have injuryStatus
                    inj = 'NA'

                proj = None
                act = None
                for stat in p['playerPoolEntry']['player']['stats']:
                    if stat['scoringPeriodId'] != week:
                        continue
                    if stat['statSourceId'] == 0:
                        act = stat['appliedTotal']
                    elif stat['statSourceId'] == 1:
                        proj = stat['appliedTotal']
                
                data.append([week, tmid, name, lineupSlot, pos, inj, proj, act])
                
    # Create DataFrame from data
    df = pd.DataFrame(data, columns=['Week','Team','Player','LineupSlot','Position','InjuryStatus','Projected','Actual'])

    return df

In [40]:
df = PullESPN(2019, swid=swid, espn_s2=espn_s2)

In [42]:
df.loc[df['LineupSlot']!=20].loc[df['Team']==1][['Week','Actual','Projected']].groupby('Week').sum()

,Actual,Projected
Week,,
1,101.24,96.655580
2,110.38,100.250947
3,107.54,100.135989
4,93.34,104.525784
5,160.24,106.565031
6,93.90,108.681849
7,0.00,104.745792
8,0.00,0.000000
9,0.00,0.000000
